In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [5]:
# 配置 Selenium
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 無頭模式，避免打開瀏覽器窗口
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
# chrome_options.add_argument('--disable-blink-features=AutomationControlled')

# 初始化 WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# 隱藏 Selenium 特徵
driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': '''
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    '''
})

{'identifier': '2'}

In [20]:
driver.get('https://www.nbc4i.com/page/29/?submit&s=intel+&orderby=modified')

In [23]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/main/section/div/h1').text == 'We’re sorry!'

True

In [6]:
url = f'https://www.nbc4i.com/?submit=&s=intel+&orderby=modified'
driver.get(url)

In [7]:
articles = driver.find_elements(By.XPATH, "//div[contains(@class, 'article-list__article-text')]//h2/a")
article_urls = [article.get_attribute('href') for article in articles]

In [10]:
driver.get(article_urls[0])

In [12]:
title = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/h1").text
author = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/div[2]/p[1]/a").text
time_element = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/div[2]/p[2]/time")
time_ = time_element.get_attribute('datetime')
paragraphs = driver.find_elements(By.XPATH, "//div[@class='article-content article-body rich-text ']//p")
content = '\n'.join([paragraph.text for paragraph in paragraphs])

articles_data = []
articles_data.append({
    'URL': url,
    'Title': title,
    'Author': author,
    'Time': time_,
    'Content': content
            })

In [26]:
keywords = ['intel', 'intel fab', 'intel foundry', 'samsung', 'samsung fab', 'samsung foundry', 'globalfoundries', 'globalfoundries fab', 'smic', 'smic fab', 'united microelectronics', 'united microelectronics fab']

# 用於保存結果的 DataFrame
articles_data = []

for keyword in tqdm(keywords):
    for page_num in range(1, 100):
        if page_num == 1:
            url = f'https://www.nbc4i.com/?submit=&s={keyword}+&orderby=modified'
        else:
            url = f'https://www.nbc4i.com/page/{page_num}/?submit&s={keyword}+&orderby=modified'
        driver.get(url)

        time.sleep(10)

        # 檢查是否有文章
        
        # 檢查是否出現 "We’re sorry!" 信息
        error_elements = driver.find_elements(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div/main/section/div/h1')
        if error_elements and error_elements[0].text == 'We’re sorry!':
            print(f"No more results for keyword: {keyword}")
            break

        # 抓取所有文章鏈接
        # /html/body/div[1]/div/div[2]/div[2]/div/main/section/div/article[13]/div/h2/a
        articles = driver.find_elements(By.XPATH, "//div[contains(@class, 'article-list__article-text')]//h2/a")
        article_urls = [article.get_attribute('href') for article in articles]
        # print(f'Found {len(article_urls)} articles for keyword: {keyword}')

        for url in tqdm(article_urls):
            try:
                driver.get(url)

                time.sleep(5)

                title = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/h1").text
                author = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/div[2]/p[1]/a").text
                time_element = driver.find_element(By.XPATH, "//html/body/div[2]/div/div[2]/div[1]/header/div/div/div[2]/p[2]/time")
                time_ = time_element.get_attribute('datetime')
                paragraphs = driver.find_elements(By.XPATH, "//div[@class='article-content article-body rich-text ']//p")
                content = '\n'.join([paragraph.text for paragraph in paragraphs])

                articles_data.append({
                    'URL': url,
                    'Title': title,
                    'Author': author,
                    'Time': time_,
                    'Content': content
                })

                df = pd.DataFrame(articles_data)
                df.to_csv('articles.csv', index=False, encoding='utf-8')
            except:
                print(f'Error occurred while scraping {url}')

        articles_data = []
        

  0%|          | 0/12 [00:00<?, ?it/s]

Error occurred while scraping https://www.nbc4i.com/celebrating-75-years-of-local-4-you-at-nbc4/


Error occurred while scraping https://www.nbc4i.com/news/ap-top-headlines/ap-japan-new-zealand-agree-on-intel-sharing-pact-amid-growing-regional-security-concerns/


Error occurred while scraping https://www.nbc4i.com/news/local-news/columbus/columbus-wants-to-install-security-camera-network-to-fight-crime-but-will-it-work/


Error occurred while scraping https://www.nbc4i.com/news/data-desk-ohio/ohio-has-one-of-the-worst-economies-in-the-country-study-says/


...

Error occurred while scraping https://www.nbc4i.com/news/local-news/central-ohio-news/nearly-20-guns-seized-during-ohio-cocaine-trafficking-arrest/


100%|██████████| 50/50 [05:46<00:00,  6.94s/it]

Error occurred while scraping https://www.nbc4i.com/news/local-news/central-ohio-news/what-are-nitazenes-the-drugs-newly-banned-in-ohio/



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/12 [14:45<?, ?it/s]


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
0   chromedriver                        0x00000001026980e8 chromedriver + 5169384
1   chromedriver                        0x000000010268ffba chromedriver + 5136314
2   chromedriver                        0x000000010220c36c chromedriver + 402284
3   chromedriver                        0x00000001021e15f4 chromedriver + 226804
4   chromedriver                        0x000000010228553d chromedriver + 898365
5   chromedriver                        0x000000010229b908 chromedriver + 989448
6   chromedriver                        0x000000010227c853 chromedriver + 862291
7   chromedriver                        0x000000010224c5c6 chromedriver + 665030
8   chromedriver                        0x000000010224ce4e chromedriver + 667214
9   chromedriver                        0x000000010265ad00 chromedriver + 4918528
10  chromedriver   

In [ ]:
df = pd.DataFrame(articles_data)
df.to_csv('articles.csv', index=False, encoding='utf-8')
